<a href="https://colab.research.google.com/github/ashishgeorge1811/naturallanguage/blob/master/fakenews%7CLSTM%7CNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from pandas import Series,DataFrame

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/fakenews.csv')

In [8]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
data.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [0]:
X= data.drop('label',axis=1)

In [0]:
y = data['label']

In [12]:
print(X.shape)
print(y.shape)

(20800, 4)
(20800,)


In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [0]:
messages = X.copy()
messages.reset_index(inplace=True)

In [0]:
import nltk
nltk.download('all')

In [0]:
import re
from nltk.corpus import stopwords

In [0]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]


In [0]:
for c in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ',str( messages['title'][c]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
  

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [0]:
#initialize vocabulary size
vocabulary_size = 5000

In [29]:
#one hot reresentation

onehot =[one_hot(words,vocabulary_size)for words in corpus]
onehot

[[1498, 3303, 814, 2858, 1662, 1690, 4694, 3875, 3757, 1109],
 [2907, 2741, 2128, 1082, 150, 2357, 4886],
 [4268, 4353, 3188, 3844],
 [2779, 1273, 2282, 3115, 3705, 2887],
 [1232, 150, 1456, 4048, 152, 4874, 150, 1341, 1313, 2184],
 [4966,
  16,
  4665,
  3258,
  96,
  4544,
  1495,
  4397,
  561,
  4708,
  3727,
  4990,
  2797,
  3956,
  4886],
 [3512, 3512, 792, 4293, 4440, 1517, 4182, 3837, 3233, 166, 2147, 2086],
 [4609, 2031, 4886, 3201, 3093, 1649, 4636, 1276, 3239, 2864, 502],
 [3067, 1150, 3230, 4189, 4544, 3435, 1361, 121, 4078, 3193, 3239, 2864, 502],
 [2241, 2627, 3090, 3237, 1078, 1426, 4544, 1828, 3239, 2864, 502],
 [4316, 1772, 635, 3358, 2937, 3396, 33, 1612, 4544, 2815],
 [2137, 1016, 2487, 1300, 4125, 3482, 3004, 1230],
 [4822, 1812, 258, 23, 878, 568, 491, 452, 842, 60, 365],
 [3115, 3387, 1662, 3396, 4544, 1078],
 [814, 3530, 816, 952, 2227, 4924, 3713, 615, 3035],
 [1552, 219, 817, 985, 3695, 2389, 2046, 3239, 2864, 502],
 [3802, 4541, 3700, 4920, 3176, 3239, 2864, 

In [0]:
#sentence length are different to make fixed perform pad_sequences(embedding sequences)

In [0]:
#Embedding representation

In [30]:
sentence_length = 20
embedding = pad_sequences(onehot,padding = 'pre',maxlen = sentence_length)  #if 'post' is replaced  in "pre" it will add 0s at the end 
print(embedding)

[[   0    0    0 ... 3875 3757 1109]
 [   0    0    0 ...  150 2357 4886]
 [   0    0    0 ... 4353 3188 3844]
 ...
 [   0    0    0 ... 3239 2864  502]
 [   0    0    0 ... 4205  164  279]
 [   0    0    0 ... 2048 3741 2661]]


In [35]:
len(embedding)

20800

In [38]:
# creating model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sentence_length))
model.add(LSTM(100))  #lstm with 100 nuerons
model.add(Dense(1,activation= 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
import numpy as np
X_final =np.array(embedding)
y_final = np.array(y)

In [40]:
print(X_final.shape)
print(y_final.shape)

(20800, 20)
(20800,)


In [0]:
#Train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test  = train_test_split(X_final,y_final,test_size = 0.33,random_state = 42)

In [42]:
#Model training

model.fit(X_train,y_train,validation_data =(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 7s 34ms/step - loss: 0.3284 - accuracy: 0.8504 - val_loss: 0.1951 - val_accuracy: 0.9197
Epoch 2/10
218/218 [==============================] - 7s 32ms/step - loss: 0.1386 - accuracy: 0.9450 - val_loss: 0.1864 - val_accuracy: 0.9248
Epoch 3/10
218/218 [==============================] - 7s 32ms/step - loss: 0.0883 - accuracy: 0.9689 - val_loss: 0.2064 - val_accuracy: 0.9238
Epoch 4/10
218/218 [==============================] - 7s 32ms/step - loss: 0.0551 - accuracy: 0.9816 - val_loss: 0.2558 - val_accuracy: 0.9234
Epoch 5/10
218/218 [==============================] - 7s 32ms/step - loss: 0.0365 - accuracy: 0.9884 - val_loss: 0.3110 - val_accuracy: 0.9180
Epoch 6/10
218/218 [==============================] - 7s 32ms/step - loss: 0.0204 - accuracy: 0.9944 - val_loss: 0.3360 - val_accuracy: 0.9142
Epoch 7/10
218/218 [==============================] - 7s 32ms/step - loss: 0.0149 - accuracy: 0.9961 - val_loss: 0.4220 - val_accuracy: 0.9126

In [0]:
y_pred =model.predict_classes(X_test)


In [46]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy = accuracy_score(y_pred,y_test)
accuracy

0.9138986013986014

In [48]:
cm =confusion_matrix(y_pred,y_test)
cm

array([[3089,  231],
       [ 360, 3184]])

In [0]:
#ADDITIONAL

In [0]:
#if needed to add drop out in the model

# Adding Dropout
from tensorflow.keras.layers import Dropout

In [51]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sentence_length))
model.add(Dropout(0.3))
model.add(LSTM(100))  #lstm with 100 nuerons
model.add(Dropout(0.3))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None
